In [ ]:
!pip install pyspark

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as t

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from geopy.distance import geodesic
from math import sqrt

In [ ]:
spark = SparkSession.builder.getOrCreate()
spark

In [ ]:
trips = spark.read.format('csv').option('header', 'true').load("trip.csv")


**Найти велосипед с максимальным временем пробега**

---



In [ ]:
trips = spark.read.format('csv').option('header', 'true').load("trip.csv")

# Группировка по 'bike_id' и вычисление максимальной длительности одной поездки
max_duration_per_bike = (
    trips
    .filter(F.col("duration").isNotNull())  # Исключаем null в duration
    .groupBy('bike_id')
    .agg(F.sum(F.col("duration").cast(t.LongType())).alias("duration"))
)

# Нахождение велосипеда с самой длинной поездкой
top_longest_trip = max_duration_per_bike.orderBy(F.col('duration').desc(),F.col('bike_id').asc())
top_longest_trip.show(1)


+-------+--------+
|bike_id|duration|
+-------+--------+
|    535|18611693|
+-------+--------+
only showing top 1 row



**Найти наибольшее геодезическое расстояние между станциями**

---



In [ ]:
stations = spark.read.format('csv').option('header', 'true').load("station.csv")
# Выбираем необходимые столбцы ('id', 'lat', 'long') из DataFrame stations
station_locations = stations.select("id", "lat", "long")
station_locations.show(5)

# Создаем комбинации станций (A и B), исключая совпадающие пары
station_pairs = (
    station_locations
    .selectExpr("id as station_A", "lat as lat_A", "long as long_A")
    .join(
        station_locations.selectExpr("id as station_B", "lat as lat_B", "long as long_B")
    )
)

# Исключаем дубликаты (оставляем только пары, где A ≠ B)
unique_station_pairs = station_pairs.filter(F.col("station_A") != F.col("station_B"))

# Функция для вычисления евклидова расстояния между двумя координатами
def compute_euclidean_distance(lat1, lon1, lat2, lon2):
    return sqrt((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2)

# Применяем функцию к каждой паре станций, создавая RDD с расстояниями
distance_rdd = unique_station_pairs.rdd.map(
    lambda row: (
        row.station_A,
        row.station_B,
        compute_euclidean_distance(
            float(row.lat_A), float(row.long_A), float(row.lat_B), float(row.long_B)
        )
    )
)

# Поиск максимального расстояния между станциями
farthest_stations = distance_rdd.max(lambda row: row[2])

# Вывод результата
farthest_stations

+---+------------------+-------------------+
| id|               lat|               long|
+---+------------------+-------------------+
|  2|         37.329732|-121.90178200000001|
|  3|         37.330698|        -121.888979|
|  4|         37.333988|        -121.894902|
|  5|         37.331415|          -121.8932|
|  6|37.336721000000004|        -121.894074|
+---+------------------+-------------------+
only showing top 5 rows



('16', '60', 0.7058482821754397)

**Найти путь велосипеда с максимальным временем пробега через станции**


---



In [ ]:
# Выбор нужных столбцов, включая 'duration', фильтрация по bike_id = 535 и поиск максимальной длительности
max_trip = (
    trips.select("id", "bike_id", "start_station_id", "end_station_id", "duration")
    .filter(F.col("bike_id") == 535)
    .orderBy(F.col("duration").cast(t.IntegerType()).desc())  # Сортировка по длительности
    .limit(1)  # Берем только одну запись с максимальной длительностью
)

# Вывод результата
max_trip.show()
print("Путь велосипеда с максимальным временем пробега:")
print(f"ID поездки: {max_trip.first()['id']}, Начальная станция: {max_trip.first()['start_station_id']}, Конечная станция: {max_trip.first()['end_station_id']}, Длительность: {max_trip.first()['duration']} секунд")


+------+-------+----------------+--------------+--------+
|    id|bike_id|start_station_id|end_station_id|duration|
+------+-------+----------------+--------------+--------+
|568474|    535|              66|            62|17270400|
+------+-------+----------------+--------------+--------+

Путь велосипеда с максимальным временем пробега:
ID поездки: 568474, Начальная станция: 66, Конечная станция: 62, Длительность: 17270400 секунд


**Найти количество велосипедов в системе**


---



In [ ]:
unique_bikes_count = trips.select("bike_id").distinct().count()
unique_bikes_count


700

**Найти пользователей потративших на поездки более 3 часов**


---



In [ ]:
# Группировка по 'zip_code' и вычисление суммарной продолжительности поездок
output_filtered = (
    trips
    .groupBy('zip_code')
    .agg(
        F.sum(F.col("duration").cast(t.IntegerType())).alias("total_duration")  # Сумма вместо максимума
    )
    .filter(F.col("total_duration") >= 10800)  # Фильтрация по суммарному времени ≥ 3 часов
)

# Подсчёт количества пользователей и вывод результата
num_users = output_filtered.count()
output_filtered.show(truncate=False)

print(f"Найдено {num_users} пользователей, потративших на поездки более 3 часов")

+--------+--------------+
|zip_code|total_duration|
+--------+--------------+
|94102   |19128021      |
|95134   |728023        |
|84606   |95145         |
|80305   |180906        |
|60070   |28919         |
|95519   |30303         |
|43085   |11670         |
|91910   |50488         |
|77339   |13713         |
|48063   |13755         |
|85022   |12682         |
|1090    |20391         |
|2136    |16010         |
|11722   |24331         |
|95138   |155365        |
|94610   |3630628       |
|94404   |3589350       |
|80301   |152189        |
|91326   |65885         |
|90742   |10965         |
+--------+--------------+
only showing top 20 rows

Найдено 3662 пользователей, потративших на поездки более 3 часов
